# Loading SageMaker Linear Learner Model with Apache MXNet in Python
> How to load SageMaker Linear Learner model with Apache MXNet in Python.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, apache, mxnet, gluon]
- image: images/copied_from_nb/images/2022-07-05-aws-linear-learner-apache-mxnet-python.jpeg

![](images/2022-07-05-aws-linear-learner-apache-mxnet-python.jpeg)

# About

You have trained a model with Amazon SageMaker builtin algorithm [Linear Learner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html). You can test this model by deploying it on a SageMaker endpoint. But you want to test this model in your local environment. In this post we will learn to use Apache MXNet and Gluon API to load the model, extract its parameters, and perform predictions locally.

# Introdcution
Apache MXNet is a fully featured, flexibly programmable, and ultra-scalable deep learning framework supporting state of the art in deep learning models, including convolutional neural networks (CNNs) and long short-term memory networks (LSTMs). Amazon has also selected MXNet as their deep learning framework of choice (see [Werner Vogels blog post](https://www.allthingsdistributed.com/2016/11/mxnet-default-framework-deep-learning-aws.html)). When you train a model using Amazon SageMaker builtin algorithm then there are high chances that the model has been trained and saved using MXNet framework. If a model has been saved with MXNet then we can use the same library to load and extract information from that model in a local environmenet.

In my last post I had used SageMaker builtin Linear Learner algorithm to train a model on Boton housing dataset. One the training was complete the trained model artifacts were stored on the S3 bucket at the following location

```
s3://sagemaker-us-east-1-801598032724/2022-06-08-sagemaker-training-overview/output/linear-learner-2022-06-16-09-04-57-576/output/model.tar.gz
```

We will use MXNet Gluon API to load this model in our local environment, extract parameters, and make some predictions. 

# Environment
This notebook is prepared with Amazon SageMaker Studio using Python 3 (Data Science) Kernel and ml.t3.medium instance.

# Loading a SageMaker Linear Learner model with Apache MXNet in Python